## XBRL US API - Python example  
This notebook contains example Python code to use the XBRL US API (https://xbrl.us/home/use/xbrl-api/)    
  
**Made by:** [Ties de Kok](https://www.tiesdekok.com)

## Imports

In [1]:
import os, re, sys, json
import requests

In [2]:
import pandas as pd
import numpy as np

In [3]:
import getpass

## Api documentation

https://xbrlus.github.io/xbrl-api/#/Facts/getFactDetails

## Generate an access token
The access token will expire after 60 minutes.

**You have two options:**

**Option 1:** obtain your own credentials

1. Obtain credentials here: https://xbrl.us/home/use/xbrl-api-community/#provisioning
2. Update 'login_cred.json' with your `client_id`, `client_secret`, and `username`  
3. Set `USE_TEMP_CRED` to False'
4. Input your password when asked

**Option 2:** Use my temporary credentials for demo purposes only

1. Set `USE_TEMP_CRED` to False
2. Input your email address when asked


In [4]:
USE_TEMP_CRED = True
if USE_TEMP_CRED:
    user_email = input(prompt="Please type your email address here: ")
    access_token = requests.get('https://tdekok-xbrlapi.builtwithdark.com/gettoken?platform=aaa-{}'.format(user_email)).text.replace('"', "")
else:
    endpoint = 'https://api.xbrl.us'
    endpoint_auth = endpoint + '/oauth2/token'

    with open('login_cred.json', 'r') as f:
        login_cred = json.loads(f.read())
    password = getpass.getpass(prompt = 'Password: ')
    body_auth = {'grant_type' : 'password', 
                'client_id' : login_cred['client_id'], 
                'client_secret' : login_cred['client_secret'], 
                'username' : login_cred['username'] ,
                'password' : password,
                'platform' : 'pc'}
    res = requests.post(endpoint_auth, data=body_auth)
    auth_json = res.json()
    access_token = auth_json['access_token']

Password: ········


## Make a query

### Define the fields you would like returned

In [5]:
fields = ['entity.cik',
         'entity.name.sort(ASC)',
         'dts.id',
         'fact.id',
         'report.filing-date',
         'period.fiscal-year',
         'report.type',
         'concept.local-name',
         'dimensions.count',
         'dimension.local-name.sort(ASC)',
         'member.local-name',
         'fact.value',
         'unit',
         'fact.decimals'
         'dimension.namespace',
         'member.namespace',
          'fact.has-dimensions'
         ]

### Define the XBRL elements (tags) you'd like 

In [6]:
XBRL_Elements = [
    'Assets',
    'Liabilities',
    'Revenues',
    'SalesRevenueNet',
    'SalesRevenueGoodsNet',
    'TotalRevenuesAndOtherIncome',
    'IntangibleAssetsNetExcludingGoodwill',
     'FiniteLivedIntangibleAssetsNet',
     'IndefiniteLivedIntangibleAssetsExcludingGoodwill'
                ]

### Define the companies you'd like

In [7]:
companies_cik = ['0000789019', ## Microsoft (MSFT)
                 '0001018724', ## Amazon (AMZN)
                 '0001652044', ## Alphabet (GOOG)
                 '0000051143', ## IBM (IBM)
                ]

### Define the years you'd like

In [8]:
#years = ['2019'] ## Use commas between for multiple years, e.g., '2018','2019'
years = [str(2000 + i) for i in range(20)] ## Years 2000 to 2020

### Specify if you want dimensions, no dimension, or all values

In [9]:
has_dimensions = 'ALL'  ## TRUE for require dimensions, FALSE for no dimensions, ALL for all values

### Specify the report types that you want

In [10]:
report_types = ['10-K', '10-K/A']

## Execute query

Note: as of 2021 it appears that the `report.type` parameter is no longer supported. Updated to filter out the document types after the data has been retrieved.

In [11]:
search_endpoint = 'https://api.xbrl.us/api/v1/fact/search'

In [12]:
params = {'concept.local-name': ','.join(XBRL_Elements),
     'period.fiscal-period': 'Y',
     'period.fiscal-year': ','.join(years),
     'unit': 'USD',
     'entity.cik': ','.join(companies_cik),
     }  
if has_dimensions == 'ALL':
    dimension_options = ['TRUE', 'FALSE']
else:
    dimension_options = [has_dimensions]
    
all_res_list = []
for dimensions_param in dimension_options:
    
    ### Every request will return a max of 2000 results. So we loop until all results are retrieved. 
    done_retrieving_all_results = False
    offset = 0
    while not done_retrieving_all_results:
        params['fact.has-dimensions'] = dimensions_param
        params['fields'] = ','.join(fields) + ',fact.offset({})'.format(offset) 
        res = requests.get(search_endpoint, params=params, headers={'Authorization' : 'Bearer {}'.format(access_token)})
        
        ## Interpret as JSON
        res_json = res.json()
            
        ## Get the results
        ### Retrieve the data list
        res_list = res_json['data']
        
        ### Keep only the request document types
        res_list = [result for result in res_list if result['report.type'] in report_types]
        
        ### Add to the results
        all_res_list += res_list
        
        ## Pagination check
        paging_dict = res_json['paging']
        print('Number of records retrieved: ', paging_dict['count'])
        if paging_dict['count'] >= 2000:
            offset += paging_dict['count']
        else:
            done_retrieving_all_results = True
    
## Convert to a DataFrame
res_df = pd.DataFrame(all_res_list)

Number of records retrieved:  2000
Number of records retrieved:  1995
Number of records retrieved:  753


## Show results

In [13]:
res_df.sample(10)

,entity.cik,entity.name,dts.id,fact.id,report.filing-date,period.fiscal-year,report.type,concept.local-name,dimensions.count,dimension.local-name,member.local-name,fact.value,unit,member.namespace,fact.has-dimensions
1046,0000051143,INTERNATIONAL BUSINESS MACHINES CORP,316710,195736105,2019-02-26,2016,10-K,Assets,2,ConsolidationItemsAxis,OperatingSegmentsMember,24085000000,USD,http://fasb.org/us-gaap/2018-01-31,True
2240,0000051143,INTERNATIONAL BUSINESS MACHINES CORP,180044,145032428,2017-02-28,2016,10-K,Revenues,2,StatementBusinessSegmentsAxis,GlobalBusinessServicesMember,121000000,USD,http://www.ibm.com/20161231,True
3573,0000789019,MICROSOFT CORPORATION,259243,155254650,2017-08-02,2016,10-K,SalesRevenueNet,0,,,85320000000,USD,,False
2331,0000051143,INTERNATIONAL BUSINESS MACHINES CORPORATION,390929,235661356,2020-02-25,2019,10-K,Revenues,2,ConsolidationItemsAxis,OperatingSegmentsExcludingIntersegmentEliminat...,27361000000,USD,http://www.ibm.com/20191231,True
2139,0000051143,INTERNATIONAL BUSINESS MACHINES CORP,278881,168979225,2018-02-27,2016,10-K,Revenues,2,StatementBusinessSegmentsAxis,TechnologyServicesCloudPlatformsMember,24311000000,USD,http://www.ibm.com/20171231,True
3526,0000789019,MICROSOFT CORPORATION,73772,48671820,2013-07-30,2012,10-K,Assets,0,,,121271000000,USD,,False
2652,0000789019,MICROSOFT CORP,28122,14257958,2011-07-28,2011,10-K,SalesRevenueNet,1,ProductOrServiceAxis,ConsultingAndProductSupportServicesMember,3372000000,USD,http://www.microsoft.com/20110630,True
628,0000051143,INTERNATIONAL BUSINESS MACHINES CORP,316710,195733741,2019-02-26,2018,10-K,Revenues,3,ConcentrationRiskByBenchmarkAxis,SalesRevenueNetMember,42024000000,USD,http://fasb.org/us-gaap/2018-01-31,True
1267,0000051143,INTERNATIONAL BUSINESS MACHINES CORP,278881,168982604,2018-02-27,2017,10-K,Revenues,2,ProductOrServiceAxis,ServicesMember,16004000000,USD,http://www.ibm.com/20171231,True
2497,0000051143,INTERNATIONAL BUSINESS MACHINES CORPORATION,390929,235659099,2020-02-25,2017,10-K,Revenues,2,StatementBusinessSegmentsAxis,GlobalBusinessServicesMember,-363000000,USD,http://www.ibm.com/20191231,True


## Show dimensions example (if exists)

In [14]:
res_df[res_df['dimensions.count'] > 1].sort_values('dts.id')[['entity.name', 'dts.id', 'period.fiscal-year', 
                                                              'report.type', 'concept.local-name','dimensions.count', 'fact.has-dimensions',
                                                             'dimension.local-name', 'member.local-name', 'fact.value', 'unit']].head(10)

,entity.name,dts.id,period.fiscal-year,report.type,concept.local-name,dimensions.count,fact.has-dimensions,dimension.local-name,member.local-name,fact.value,unit
698,INTERNATIONAL BUSINESS MACHINES CORP,24315,2009,10-K,Revenues,2,True,ConcentrationRiskByTypeAxis,GeographicConcentrationRiskMember,51386000000,USD
2253,INTERNATIONAL BUSINESS MACHINES CORP,24315,2009,10-K,Revenues,2,True,StatementGeographicalAxis,US,34150000000,USD
2259,INTERNATIONAL BUSINESS MACHINES CORP,24315,2009,10-K,Revenues,2,True,StatementGeographicalAxis,JP,10222000000,USD
2263,INTERNATIONAL BUSINESS MACHINES CORP,24315,2009,10-K,Revenues,2,True,StatementGeographicalAxis,OtherCountriesMember,51386000000,USD
2251,INTERNATIONAL BUSINESS MACHINES CORP,24315,2011,10-K,Revenues,2,True,StatementGeographicalAxis,OtherCountriesMember,58906000000,USD
2297,INTERNATIONAL BUSINESS MACHINES CORP,24315,2011,10-K,Revenues,2,True,StatementGeographicalAxis,JP,10968000000,USD
670,INTERNATIONAL BUSINESS MACHINES CORP,24315,2011,10-K,Revenues,2,True,ConcentrationRiskByTypeAxis,GeographicConcentrationRiskMember,37041000000,USD
2289,INTERNATIONAL BUSINESS MACHINES CORP,24315,2010,10-K,Revenues,2,True,StatementGeographicalAxis,OtherCountriesMember,53589000000,USD
2283,INTERNATIONAL BUSINESS MACHINES CORP,24315,2010,10-K,Revenues,2,True,StatementGeographicalAxis,JP,10701000000,USD
2279,INTERNATIONAL BUSINESS MACHINES CORP,24315,2011,10-K,Revenues,2,True,StatementGeographicalAxis,US,37041000000,USD
